In [3]:
# librerias
import pandas as pd

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mkechinov/ecommerce-behavior-data-from-multi-category-store")

print("Path to dataset files:", path)

 35%|███▍      | 1.49G/4.29G [02:18<04:21, 11.5MB/s]


KeyboardInterrupt: 

In [7]:

# Path del dataset
dataset_path = "C:/Users/Joabh/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8/2019-Nov.csv"

# Tamaño objetivo de la muestra
target_sample_size = 25000

# Lectura en chunks, muestreo aleatorio y acumulación
sampled_data = []  # Lista para guardar las muestras
chunk_size = 10**6  # Tamaño del chunk

# Iteramos sobre cada chunk del CSV
for chunk in pd.read_csv(dataset_path, chunksize=chunk_size, parse_dates=['event_time']):
    # Convertir event_time a datetime y extraer la hora
    chunk['hour'] = chunk['event_time'].dt.hour
    
    # Muestreo estratificado por hora (cada hora estará representada proporcionalmente)
    chunk_sample = chunk.groupby('hour').apply(lambda x: x.sample(frac=0.01, random_state=1)).reset_index(drop=True)
    sampled_data.append(chunk_sample)
    
    # Si ya hemos alcanzado o superado el tamaño de la muestra objetivo, rompemos el ciclo
    if sum(len(sample) for sample in sampled_data) >= target_sample_size:
        break

# Concatenamos las muestras en un DataFrame
final_sample = pd.concat(sampled_data)

# Si tenemos más de 25k filas, tomamos una muestra aleatoria final
if len(final_sample) > target_sample_size:
    final_sample = final_sample.sample(n=target_sample_size, random_state=1)

# Guardar la muestra final a un archivo CSV
output_path = "C:/Users/Joabh/Documents/GitHub/ProyectoFInal_DS/sampled_data_25k.csv"
final_sample.to_csv(output_path, index=False)

print(f"Se ha generado una muestra aleatoria de {len(final_sample)} filas y se ha guardado en {output_path}.")




C:\Users\Joabh\AppData\Local\Temp\ipykernel_20772\2722909316.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chunk_sample = chunk.groupby('hour').apply(lambda x: x.sample(frac=0.01, random_state=1)).reset_index(drop=True)
C:\Users\Joabh\AppData\Local\Temp\ipykernel_20772\2722909316.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chunk_sample = chunk.groupby('hour').apply(lambda x: x.sample(frac=0.

Se ha generado una muestra aleatoria de 25000 filas y se ha guardado en C:/Users/Joabh/Documents/GitHub/ProyectoFInal_DS/sampled_data_25k.csv.


In [2]:
import pandas as pd

df = pd.read_csv("folder/2019-Nov.csv")

# Convertir 'event_time' a tipo datetime, si no lo está
df['event_time'] = pd.to_datetime(df['event_time'])

# Definir el intervalo de tiempo para la reducción (por ejemplo, cada 5 minutos)
interval = '5T'  # 'T' representa minutos en pandas

# Agrupar por intervalos de 5 minutos y por tipo de evento
df_reduced = df.set_index('event_time').groupby([pd.Grouper(freq=interval), 'event_type']).size().unstack(fill_value=0)

# Aplanar los datos para que cada intervalo de tiempo tenga una fila representativa por evento
df_reduced = df_reduced.reset_index()

# Mostrar un resumen de los primeros registros del dataframe reducido
print(df_reduced.head())

# Guardar el dataframe reducido a un archivo CSV
df_reduced.to_csv("folder/reduced_data.csv", index=False)


MemoryError: Unable to allocate 1.51 GiB for an array with shape (3, 67501979) and data type int64

In [1]:
import pandas as pd

# Configuración
input_file = 'folder/2019-Nov.csv'  # Ruta de tu archivo de datos
output_file = 'reduced_dataset.csv'  # Archivo de salida con datos reducidos
chunk_size = 10**6  # Fragmentos de un millón de filas para procesar en cada iteración

# Lista para almacenar fragmentos procesados
processed_data = []

# Procesamiento en fragmentos
for chunk in pd.read_csv(input_file, chunksize=chunk_size, parse_dates=['event_time']):
    # Redondea cada evento al inicio del intervalo de 5 minutos
    chunk['5min_interval'] = chunk['event_time'].dt.floor('5T')
    
    # Contar eventos por tipo en cada intervalo de 5 minutos
    summarized_chunk = chunk.groupby(['5min_interval', 'event_type']).size().unstack(fill_value=0)

    # Añadir el fragmento procesado a la lista
    processed_data.append(summarized_chunk)

# Concatenar todos los fragmentos procesados
reduced_data = pd.concat(processed_data)
reduced_data = reduced_data.groupby(reduced_data.index).sum()  # Consolidar duplicados

# Guardar el conjunto de datos reducido
reduced_data.to_csv(output_file)
print(f"Datos reducidos guardados en {output_file}")


Datos reducidos guardados en reduced_dataset.csv
